# Assignment 3

Due March 20th at **10am**.  Pre-grading will be posted on CourseSpaces once the script is running.  Please save your assignment notebook in your `mp248` repo as `mp248/Assignment.3/Assignment.3.ipynb`. Please keep all output and data files in that  `mp248/Assignment.3` notebook. Add all files that you are asked to generate to your git commit, except the nuclear data files.

For each plot that you are asked to make create a `.png` image and add it to your commit. Put the problem number and sub-number in the file name, such as `P1.4.png`.

Problem 1 leans heavily on material done in Course Notebook 11 and Lab 11.a and 11.b. In the week March 11 -15 we will provide all the usual help in the labs regarding questions concerning Lab 11.a and 11.b and  Course Notebook 11. However, in the lab March 18th we will not be answering any questions concerning those labs or the assignment to treat students in the Monday and Wednesday lab the same. 

As a tip, it is strongly recommended to completely finish the labs 11.a and 11.b, as this will be of great help. 

## 1 Temperature-dependent network solution

The goal of this problem is to extend the network solution from Course Notebook 11 to allow the network integration to follow a time-dependent temperature evolution (trajectory). Different T mean different rates in the RHS. 

#### 1.1 Network with constant rates
- Collect the **essential** code components that are required to solve the nuclear network as described in Course Notbook 11, using the `integrate.odeint`, using the rates for $T9=0.09$, $\rho$ and initial abundances as in the course notebook. 
- Make a plot of the evolution of the mass fractions (`Y*A`) as a function of time in the time interval `[0,1.e6]`s. Make sure all lines have different linestyle and color, as well as a legend. 
- Open a  new file called `results.txt` using `open` (check the docstring for the right `mode` option) and write the mass fraction of $^{15}N$ (`n15`) at $t=10,000s$  and $10^6$ in the first two lines of the file, at the end of a formatted statement which each say: `The N15 abundance at t=xxx is: ...` (replace the three dots `...` with the mass fraction value and `xxx` with time in s). 

In [3]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


In [4]:
# read initial abundance file
#f.close()
f_ini=open('data/iniab1.4E-02As09.ppn')

In [5]:
ind=[];elem=[];A=[];X=[]   # we will sort the row items into these three lists
for i,line in enumerate(f_ini.readlines()):
    a,b,c,d=line.split()
    # the first column in the file contains the charge 
    ind.append(i)  # number; we don't need it, but an index variable 
    elem.append(b) # would be useful
    A.append(c)
    X.append(d)

In [6]:
# close the file handle!
f_ini.close()

In [7]:
# 
X0=array(X,float)
A=array(A,float)
Y0 = X0/A

In [8]:
global rate

rate=[7.36E-06]       # C12(p,g)
rate.append(3.52E-05) # C13(p,g)
rate.append(2.36E-07) # N14(p,g)
rate.append(2.03E-02) # N15(p,a)

rate = array(rate)
#print(rate)

In [9]:
def react_terms(y):
    terms=[]
    terms.append(rate[0]*y[2]*y[0]) # 0 C12(p,g)
    terms.append(rate[1]*y[3]*y[0]) # 1 C13(p,g)
    terms.append(rate[2]*y[4]*y[0]) # 2 N14(p,g)
    terms.append(rate[3]*y[5]*y[0]) # 3 N15(p,a)
    return array(terms)

In [10]:
def f_rhs(y,t):
    '''Provide RHS for CN network equations''' 

    terms = react_terms(y)
    
    #print(t)

    dh1_dt  =  -terms.sum()
    dhe4_dt =   terms[3]
    dc12_dt =  -terms[0] + terms[3]
    dc13_dt =  -terms[1] + terms[0]
    dn14_dt =  -terms[2] + terms[1]
    dn15_dt =  -terms[3] + terms[2]
    
    return array([dh1_dt,dhe4_dt,dc12_dt,dc13_dt,dn14_dt,dn15_dt])

In [11]:
# in order to plot the lines of all elements setup a linestyle list
linestyles = ['-', '--', '-.', ':']
# we only have 4 linestyles, with the modulus function we can get
# a linestyle for any i
i = randint(100)
#print(linestyles[mod(i,4)])

In [12]:
dt        = 10000.

In [13]:
from scipy import integrate

In [14]:
t         = arange(0,1 + 1.e6,dt)  
markevery = 1

In [15]:
Y=integrate.odeint(f_rhs,Y0,t)

In [16]:
close(1);figure(1)
for i in range(len(A)):
    plot(t,log10(Y.T[i]*A[i]),label=elem[i]+str(int(A[i])),\
         linestyle=linestyles[mod(i,4)],marker=i+5,markevery=markevery)
    
    #if i == 5:
     #   print(t)
      #  print(Y.T[i]*A[i])
    
legend(loc=4)
ylabel('mass fraction');xlabel('time/s');title('mass fraction as a function of time')
#ylim(-2e-6,2e-6)

pyplot.savefig('P1.1.png')

FigureCanvasNbAgg()

In [17]:
## The N15 abundance at t=xxx is: ...
## at t = 10,000 and 10^6

i = 5
mf = Y.T[i]*A[i]
#print(len(mf))
#print(mf[100])


file = open('results.txt','w')
file.write("The N15 abundance at t=10000 is: ")
file.write(str(mf[1]))
file.write('\n')
file.write("The N15 abundance at t=1e6 is: ")
file.write(str(mf[100]))

file.close()

#### 1.2 Trajectory file
- Read file `T-evol.dat` using numpy's `loadtxt` method and combine all data read from the file into one dictionary `traj_data`, so that you can access it like this: `traj_data['T9']` and likewise for key `'time'`. 
- Plot temperature as a function of time. Use log scale when appropriate. 
- Using python commands (don't copy and paste!) open again the file `results.txt` (assuming you have closed it previously) and add as the third line the first values of time and temperature contained in the file `T-evol.dat` using the dictionary `traj_data`. Again, check the docstring of the `write` method what `mode=` option is needed to append, and recall that the sting `'\n'` is interpreted as a newline. 

In [18]:
close(2);figure(2)
traj_data = {'time': '', 'T9': ''}
traj_data['time'],traj_data['T9'] = loadtxt('data/T-evol.dat',unpack=True,usecols=(0,1))
plot(traj_data['time'],log(traj_data['T9']))
xlabel('time')
ylabel('log(T9)')
title('T-evol data')

file = open('results.txt','a')
file.write('\n')
file.write(str(traj_data['time'][0]))
file.write(' ')
file.write(str(traj_data['T9'][0]))
file.write('\n')

file.close()

pyplot.savefig('P1.2.png')

FigureCanvasNbAgg()

#### 1.3 Trajectory interpolation
For the integration we need the temperature at any time the solver decides to request. 
- Use scipy's `interpolate.interp1d` to set up an interpolation function called `T9int`  for `T9` that returns for any time within the limits of the trajectory the  temperature and density, using the linear interpolation option. Make sure your interpolation function has the extrapolation option turned on.
- Add the output of that function for `t=1.5e+4` and `t=2.9e+7` to the file `results.txt`. 
    

In [19]:
from scipy import interpolate

T9int = interpolate.interp1d(traj_data['time'],traj_data['T9'],fill_value='extrapolate')
t1 = 1.5e+4
T91 = T9int(t1)
t2 = 2.7e+7
T92 = T9int(t2)

file = open('results.txt','a')
file.write(str(T91))
file.write(' ')
file.write(str(T92))
file.write('\n')
file.close()

#### 1.4 Nuclear data `get_rates` function
- Collect the essential code from Lab 11.a that reads the T-dependent reaction rate data and provides the function `get_rates()` to provide for a given input temperature the `rate` list required in the solution above. Be careful what unit the temperature is in. In the trajectory, as indicated in the header, the unit is plain Kelvin, but in the reaction rate files it is in units of $10^9$K, also referred to as `T9`. 
- Write the output of `str(get_rates(traj_data['T9'][0]))` as another line to the `results.txt` file. 

In [20]:
def get_rates(T):
    f = np.loadtxt('data/12cpg13n.dat')
    filenames = ['data/12cpg13n.dat','data/13cpg14n.dat',
                 'data/14npg15o.dat','data/15npa12c.dat']
    rates = list()
    for file in filenames:
        f = np.loadtxt(file)
        T9 = list() 
        adopted = list()
        low = list()
        high = list()
        i = 0
        while i < len(f):
            T9.append(f[i][0])
            adopted.append(f[i][1])
            low.append(f[i][2])
            high.append(f[i][3])
            i = i + 1
    
        alldata = [T9,adopted,low,high]
        j = 0
        k = 0
        while j < len(alldata[0]):
            if alldata[0][j] == T:
                k = j
            j = j + 1
        rates.append(alldata[1][k])
    return rates
        
rates = get_rates(traj_data['T9'][0])
print(traj_data['T9'][0])
print(rates)


file = open('results.txt','a')
file.write(str(rates))
file.write('\n')
file.close()

0.4
[3.18, 5.97, 4.77, 45800.0]


## 2 Higher-order derivative

### In course notebook 5 we introduced a first-order accurate numerical derivative. Review that course notebook material.
It is very easy to increase the order of the difference equation to second order, and thereby improve the accuracy. The idea is to take into account one more term of the Taylor expansion, then solve again for $f^\prime(x)$ as shown below:

$$
f(x+h) = f(x) + hf^\prime(x) + \frac{1}{2}h^2f^{\prime\prime}(x)
$$
 and solve for $f^\prime(x)$ 
$$
f^\prime(x)  = \frac{f(x+h) - f(x)}{h}  - \frac{1}{2}hf^{\prime\prime}(x) 
$$

with the second order derivative being approximated to first order by
$$
f^{\prime\prime}(x) \approx \frac{f'(x+h) -f'(x)}{h}
$$

* Implement a function `deriv2` which takes the same arguments as `deriv1` introduced in notebook 5, but implements the first derivative in second-order accurate discretization described above. Test it for the third-order polynomial $f(x) = x^3$ for $x=1$.
* Create a convergence test plot as in Figure 2 in notebook 5 that shows the dependence of the residual `log10 (df/dx - 3.0)` as a function of `log10(h)` where `h = 10**npow` and `npow=[0, -1, ..., -10]` for both the first-order and second-order accurate functions `derv1` and `deriv2`.

Finally, add to the `results.txt` file the line 

`Residual first- and second-order for npow=-2: ...` 

replacing the dots `...` with the two numbers at that value of h for both derivatives.

In [29]:
def deriv1(f,x,h):
    dfdx = (f(x+h) - f(x)) / h
    return dfdx

def deriv2(f,x,h):
    fp1 = (f(x+h) - f(x)) / h
    fp2 = (f(x+h+h) - f(x+h)) / h
    fpp = (fp2 - fp1)/h
    dfdx = fp1 - h*fpp/2.0
    return dfdx

f = lambda x: x**3
x = 1
h = 0.1
d1 = deriv1(f,x,1)
d2 = deriv2(f,x,1)
#print(d1)
#print(d2)

In [30]:
npow = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0]
h = list()
for x in npow:
    h.append(10**x)
    
dfdx1 = list()
y1 = list()
dfdx2 = list()
y2 = list()
for k in h:
    dfdx1.append(deriv1(f,1,k))
    dfdx2.append(deriv2(f,1,k))
    y1.append(abs(3.0 - deriv1(f,1,k)))
    y2.append(abs(3.0 - deriv2(f,1,k)))
    
#print(y1)

close(4);figure(4)
plot(log(h),log(y1), 'og', label = 'deriv1')
plot(log(h),log(y2), 'ob', label = 'deriv2')
xlim(0,-25),xlabel('log(h)'),ylabel('log(3.0 - df/dx)'),title('dependence of the residule on log(h)'),legend()


file = open('results.txt','a')
file.write("Residual first- and second-order for npow=-2: ")
file.write(str(y1[8]))
file.write(' ')
file.write(str(y2[8]))
file.write('\n')
file.close()

pyplot.savefig('P2.png')

FigureCanvasNbAgg()